# Exploratory Analysis of Import Data

This notebook explores the PIERS Bill of Lading data, obtained from S&P's Global Trade Analytics Suite. See the README.md file for more info on the overall project, data pre-processing, and column definitions. 

In [1]:
#import libraries
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns


#display settings
pd.set_option('display.max_columns', None)
%matplotlib inline

#enable string cache for polars categoricals
pl.enable_string_cache()

## Basic Summary Stats

In [2]:
#set paths
imports_path = 'data/clean/imports/'
exports_path = 'data/'

#get schema and col names
imports_schema = pl.read_parquet_schema(source= imports_path+'piers_imports_2005.parquet')
imports_colnames = imports_schema.keys()
exports_schema = pl.read_parquet_schema(source=exports_path+'piers_exports_complete.parquet')
exports_colnames = exports_schema.keys()

#init lazy dataframes
imports_lzdf = pl.scan_parquet(imports_path+'*.parquet', parallel='columns')
exports_lzdf = pl.scan_parquet(exports_path+'piers_exports_complete.parquet', parallel='columns')

#get number of observations
imports_n = imports_lzdf.select(pl.count()).collect().item()
exports_n = exports_lzdf.select(pl.count()).collect().item()
print('The imports dataset has {:,} rows and {} columns.'.format(imports_n, len(imports_schema)))
print('The exports dataset has {:,} rows and {} columns.'.format(exports_n, len(exports_schema)))

The imports dataset has 178,875,896 rows and 44 columns.
The exports dataset has 68,769,489 rows and 31 columns.


In [3]:
#view head of imports dataframe
print('Imports dataframe preview:')
imports_lzdf.limit(n=3).collect()

Imports dataframe preview:


weight,weight_unit,qty,qty_type,teus,value_est,date_arrival,container_piece_count,commod_short_desc_qty,origin_territory,origin_region,arrival_port_code,arrival_port_name,departure_port_code,departure_port_name,dest_final,coast_region,clearing_district,place_receipt,shipper_name,shipper_address,consignee_name,consignee_address,notify_party1_name,notify_party1_address,notify_party2_name,notify_party2_address,commod_desc_raw,container_id_marks,marks_desc,hs_code,joc_code,commod_short_desc,container_ids,carrier_name,carrier_scac,vessel_name,voyage_id,precarrier,vessel_id,inbond_code,transport_mode,bol_number,bol_id
f64,cat,f64,cat,f64,f64,datetime[μs],i32,str,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,str,str,str,str,str,str,str,str,str,str,str,cat,cat,str,str,cat,cat,str,str,f64,i32,f64,cat,str,str
0.0,"""KG""",0.0,"""-1""",0.0,0.0,2005-10-12 00:00:00,0,"""0""","""UNITED KINGDOM…","""NORTH EUROPE""","""1601""","""CHARLESTON""","""41351""","""THAMESPORT""",""" ""","""EAST""","""CHARLESTON, S.…","""BRIDGWATER""","""FORMASTERCONTA…","""""","""FORMASTERCONTA…","""""","""FORMASTERCONTA…","""""",null,null,null,"""""","""""","""009669""","""""","""""","""""","""HAPAG LLOYD""","""HAPL""","""CP LIBERATOR""","""8039""",null,8415952,null,null,"""LIV050931677""","""HAPL_LIV050931…"
0.0,"""KG""",1832.0,"""CTN""",0.0,0.0,2005-03-30 00:00:00,1,"""1832""","""CHINA (MAINLAN…","""NORTH EAST ASI…","""2709""","""LONG BEACH""","""58023""","""BUSAN""",""" ""","""WEST""","""LOS ANGELES, C…","""TIANJIN""","""ORDER""","""""","""ORDER""","""""","""ORDER""","""""",null,null,null,"""""","""LC""","""721590""","""""","""""","""TRLU6266377""","""MEDITERRANEAN …","""MDSC""","""MSC JENNY""","""510""",null,8709169,null,null,"""UMSCHH51015F""","""MDSC_UMSCHH510…"
0.0,"""KG""",3500.0,"""PKG""",0.0,0.0,2005-01-26 00:00:00,2,"""3500""","""SPAIN""","""MEDITERRANEAN""","""4909""","""SAN JUAN""","""47094""","""VALENCIA""",""" ""","""CARIBBEAN""",null,"""VALENCIA""","""PERFUMERIA GAL…","""AVDA DE MADRID…","""LOUIS GARRATON…","""""","""LOUIS GARRATON…","""""",null,null,null,"""""","""ZZ; ZZ""","""330300""","""""","""""","""CTEU2003642 CT…","""CIA. TRANSATLA…","""CTES""","""TMM SINALOA""","""32""",null,8406286,null,null,"""VLOPP0407045""","""CTES_VLOPP0407…"


In [4]:
#view head of exports dataframe 
print('Exports dataframe preview:')
exports_lzdf.limit(n=3).collect()

Exports dataframe preview:


shipper,shipper_address,weight,weight_unit,quantity,quantity_type,teus,carrier_name,carrier_scac,vessel_name,voyage_number,bol_number,vessel_id,value_est,departure_port_code,departure_port_name,container_ids,container_piece_count,coast_region,commod_desc_raw,commod_short_desc,hs_code,joc_code,commod_short_desc_qty,date_departure,origin,dest_territory,dest_region,dest_port_code_declared,dest_port_name,bol_id
str,str,f64,cat,f64,cat,f64,cat,cat,str,str,str,i32,f64,cat,cat,str,i32,cat,str,str,str,str,str,datetime[μs],str,cat,cat,i32,cat,str
null,null,0.0,"""KG""",17.0,"""UNT""",0.0,"""HOEGH UGLAND A…","""AUTO""","""OCEAN SPIRIT""","""93""","""03102""",8321747,0.0,"""1803""","""JACKSONVILLE""","""""",0,"""EAST""",null,"""""","""870390""","""""","""1; 2; 1; 2; 1;…",2005-12-26 00:00:00,""" ""","""SAUDI ARABIA""","""MIDDLE EAST""",51721,"""JEDDAH""","""AUTO_03102"""
null,null,0.0,"""KG""",3120.0,"""PKG""",0.0,"""MAERSK LINE""","""MLSL""","""SEA LAND INNOV…","""513""","""SJ1724618""",7820851,0.0,"""3126""","""ANCHORAGE""","""GLDU0487348 MA…",5,"""WEST""",null,"""""","""030510""","""""","""3120""",2005-09-15 00:00:00,""" ""","""JAPAN""","""NORTH EAST ASI…",58895,"""YOKOHAMA""","""MLSL_SJ1724618…"
null,null,0.0,"""KG""",1.0,"""LOT""",0.0,"""P&O NEDLLOYD""","""PONL""","""OOCL NINGBO""","""6636""","""LNAM30127882""",9256482,0.0,"""2709""","""LONG BEACH""","""INBU3009947""",1,"""WEST""",null,"""""","""4805""","""""","""1""",2005-12-14 00:00:00,""" ""","""SINGAPORE""","""SOUTH EAST ASI…",55976,"""SINGAPORE""","""PONL_LNAM30127…"


In [5]:
#Summary stats for Imports

#init df and get stats labels column
imports_summarystats_df = imports_lzdf.select(pl.first()).collect().describe().select(pl.first()).to_pandas()
#loop through columns and get descriptive stats
for column in imports_colnames:
    imports_summarystats_df[column] = imports_lzdf.select(pl.col(column)).collect().describe().select(column).to_pandas()
#display
print('Summary Stats - Imports')
imports_summarystats_df

Summary Stats - Imports


,describe,weight,weight_unit,qty,qty_type,teus,value_est,date_arrival,container_piece_count,commod_short_desc_qty,origin_territory,origin_region,arrival_port_code,arrival_port_name,departure_port_code,departure_port_name,dest_final,coast_region,clearing_district,place_receipt,shipper_name,shipper_address,consignee_name,consignee_address,notify_party1_name,notify_party1_address,notify_party2_name,notify_party2_address,commod_desc_raw,container_id_marks,marks_desc,hs_code,joc_code,commod_short_desc,container_ids,carrier_name,carrier_scac,vessel_name,voyage_id,precarrier,vessel_id,inbond_code,transport_mode,bol_number,bol_id
0,count,1.788759e+08,178875896,1.788759e+08,150054988,1.788759e+08,1.788759e+08,178875896,1.788759e+08,178875850,178004511,178004511,178842823,178842823,178399200,178399200,178875896,178767698,138267806,178028936,177152819,149352161,177316770,158522570,131500774,147349739,11190643,80229284,153344894,178875896,178875896,178875850,178875850,178875850,178875896,178743544,178875872,167484536,178875896,0.0,1.673943e+08,4.599980e+07,54698436,178875896,178875896
1,null_count,0.000000e+00,0,0.000000e+00,28820908,0.000000e+00,0.000000e+00,0,0.000000e+00,46,871385,871385,33073,33073,476696,476696,0,108198,40608090,846960,1723077,29523735,1559126,20353326,47375122,31526157,167685253,98646612,25531002,0,0,46,46,46,0,132352,24,11391360,0,178875896.0,1.148159e+07,1.328761e+08,124177460,0,0
2,mean,4.740686e+04,None,1.203557e+03,None,1.285770e+00,7.808732e+04,None,1.440886e+00,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,9.338100e+06,3.066917e+02,None,None,None
3,std,1.721627e+06,None,4.090204e+05,None,3.112699e+00,2.853759e+06,None,3.409732e+00,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,2.850288e+05,1.059749e+03,None,None,None
4,min,0.000000e+00,None,-7.456109e+07,None,0.000000e+00,0.000000e+00,2005-01-01 00:00:00,0.000000e+00,,None,None,None,None,None,None,None,None,None,None,,,,,,,& OPTCO,,,,,None,None,,,None,None,,,NaN,0.000000e+00,0.000000e+00,None,$70050200001,-1_CSHSE0009996
5,25%,0.000000e+00,None,1.000000e+00,None,0.000000e+00,0.000000e+00,None,1.000000e+00,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,9.248112e+06,0.000000e+00,None,None,None
6,50%,1.466000e+03,None,3.800000e+01,None,2.200000e-01,6.571000e+03,None,1.000000e+00,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,9.332884e+06,0.000000e+00,None,None,None
7,75%,1.501000e+04,None,4.000000e+02,None,2.000000e+00,4.552600e+04,None,1.000000e+00,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,9.462706e+06,0.000000e+00,None,None,None
8,max,6.172936e+09,None,3.531460e+09,None,1.730750e+03,9.981402e+09,2023-11-30 00:00:00,9.990000e+02,9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9...,None,None,None,None,None,None,None,None,None,None,“COOPERATIVA OPERAI CAVATORI DEL BOTTICINO,ÞORRASALIR 13,C MCCRACKEN,ŠILUTES STR 9,C MCCRACKEN,ŠILUTES STR 9,ZYP COATINGS,co RMI Titanium Company LLC 208, WORKING GLOVES,}No MarksNo MarksNo Marks,; ; ; ; ; ; ; ; ; ; ; ,None,None,"ZIRCONIUM; SAND,FLOUR,SPONGE,POWDER VALVES; EN...",vTA1108,None,None,ZYGI,^144,NaN,9.975363e+06,7.122000e+03,None,_YLC158932,zimu_ZIMUZHJ0000952


In [6]:
#Summary stats for Exports

#NOTE at the moment the exports dataset may fit in memory, in which case the below code could be accomplished more efficiently
#by executing the following line; however, the main code below should run even when the dataset does not fit in memory. 
#exports_lzdf.collect().describe()

#init df and get stats labels column
exports_summarystats_df = exports_lzdf.select(pl.first()).collect().describe().select(pl.first()).to_pandas()
#loop through columns and get descriptive stats
for column in exports_colnames:
    exports_summarystats_df[column] = exports_lzdf.select(pl.col(column)).collect().describe().select(column).to_pandas()
#display
print('Summary Stats - Exports')
exports_summarystats_df

Summary Stats - Exports


,describe,shipper,shipper_address,weight,weight_unit,quantity,quantity_type,teus,carrier_name,carrier_scac,vessel_name,voyage_number,bol_number,vessel_id,value_est,departure_port_code,departure_port_name,container_ids,container_piece_count,coast_region,commod_desc_raw,commod_short_desc,hs_code,joc_code,commod_short_desc_qty,date_departure,origin,dest_territory,dest_region,dest_port_code_declared,dest_port_name,bol_id
0,count,52901485,51534703,6.876949e+07,68769489,6.876949e+07,61810793,6.876949e+07,68595453,68768115,63384893,67625608,68769456,6.323115e+07,6.876949e+07,68720145,68720145,68769489,6.876949e+07,68769013,53168182,68769437,68769437,68769437,68769437,68769489,68769489,68279599,68279599,6.829928e+07,68299282,68769456
1,null_count,15868004,17234786,0.000000e+00,0,0.000000e+00,6958696,0.000000e+00,174036,1374,5384596,1143881,33,5.538337e+06,0.000000e+00,49344,49344,0,0.000000e+00,476,15601307,52,52,52,52,0,0,489890,489890,4.702070e+05,470207,33
2,mean,None,None,1.152067e+05,None,3.965311e+04,None,1.672979e+00,None,None,None,None,None,9.159625e+06,1.415664e+05,None,None,None,1.923483e+00,None,None,None,None,None,None,None,None,None,None,4.907175e+04,None,None
3,std,None,None,2.375519e+06,None,3.471335e+06,None,5.518306e+00,None,None,None,None,None,5.438769e+05,6.466967e+06,None,None,None,1.109810e+01,None,None,None,None,None,None,None,None,None,None,1.906010e+04,None,None
4,min,,,0.000000e+00,None,-1.467123e+09,None,0.000000e+00,None,None,,,,1.960000e+02,0.000000e+00,None,None,,0.000000e+00,None,,,,,,2005-01-01 00:00:00,,None,None,0.000000e+00,None,079A_26004878070
5,25%,None,None,0.000000e+00,None,0.000000e+00,None,0.000000e+00,None,None,None,None,None,9.143568e+06,0.000000e+00,None,None,None,1.000000e+00,None,None,None,None,None,None,None,None,None,None,3.370000e+04,None,None
6,50%,None,None,1.360800e+02,None,1.300000e+01,None,0.000000e+00,None,None,None,None,None,9.295220e+06,4.080000e+02,None,None,None,1.000000e+00,None,None,None,None,None,None,None,None,None,None,5.205100e+04,None,None
7,75%,None,None,1.919696e+04,None,1.780000e+02,None,2.000000e+00,None,None,None,None,None,9.398424e+06,4.627100e+04,None,None,None,1.000000e+00,None,None,None,None,None,None,None,None,None,None,5.820100e+04,None,None
8,max,SUNSET TRANSPORTATION SERVICE,,9.529981e+08,None,3.716633e+09,None,3.729250e+03,None,None,xin yang shan,|SAL5,PE10518042,9.980629e+06,1.000000e+10,None,None,~POPLAR AVE |SAVALY762976 _38197 WFHU5153172 W...,4.136000e+03,None,ÙTEEL GUARD RAIL; ÙTEEL GUARD RAILS,"ZIRCONIUM; SAND,FLOUR,SPONGE,POWDER ZIRCONIUM ...",ddedo,9669000 7746000 1301000 6830000,9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9; 9...,2023-11-30 00:00:00,,None,None,9.993000e+04,None,zzzz_ZZZZ


In [13]:
#Counts of top imports categorical columns

#get categorical cols
imports_catcols_lzdf = (
    imports_lzdf.select(pl.col(pl.Categorical))
)
#init top cats df
imports_topcats_df = pl.DataFrame()

for colname in imports_catcols_lzdf.columns:
    col_counts_df = (
        #select column
        imports_catcols_lzdf.select(pl.col(colname))
        .collect()
        .to_series()
        #get top 10 value counts
        .value_counts()
        .sort('count', descending=True)
        .limit(10)
        #rename count column
        .rename({'count': colname + '_count'})
    )
    #stack dfs horizontally 
    imports_topcats_df = pl.concat([imports_topcats_df, col_counts_df], how='horizontal')

print('Counts of top 10 categories for each categorical variable in imports database:')
imports_topcats_df

Counts of top 10 categories for each categorical variable in imports database:


weight_unit,weight_unit_count,qty_type,qty_type_count,origin_territory,origin_territory_count,origin_region,origin_region_count,arrival_port_code,arrival_port_code_count,arrival_port_name,arrival_port_name_count,departure_port_code,departure_port_code_count,departure_port_name,departure_port_name_count,dest_final,dest_final_count,coast_region,coast_region_count,clearing_district,clearing_district_count,place_receipt,place_receipt_count,hs_code,hs_code_count,joc_code,joc_code_count,carrier_name,carrier_name_count,carrier_scac,carrier_scac_count,transport_mode,transport_mode_count
cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32
"""KG""",178875896,"""PKS""",1,"""NAURU""",1,"""TBD""",7,"""47000""",1,"""XIAMEN""",1,"""56035""",1,"""BOSPORUS""",1,""" """,178875896,null,108198,"""DALLAS/FORT WO…",4,"""TAUMARUNUI""",1,"""850440 8544 85…",1,"""6921000 661105…",1,"""NOBLE SHPG INC…",1,"""DYFA""",1,"""MARITIME""",54698436
null,null,"""TIE""",1,"""COOK ISLANDS""",1,"""MIDDLE EAST""",652610,"""4107""",1,"""MANZANILLO PA""",1,"""47582""",1,"""NEGISHI""",1,null,null,"""GREAT LAKES""",134208,"""OGDENSBURG, NE…",164,"""KARMSUND""",1,"""852031 852692 …",1,"""7275000 349100…",1,"""HERM DAUELSBER…",1,"""YMAI""",1,null,124177460
null,null,"""STM""",1,"""BRITISH INDIAN…",2,"""AFRICA""",690351,"""2721""",1,"""BATH ME""",1,"""20551""",1,"""AIROLA""",1,null,null,"""CARIBBEAN""",1486746,"""MINNEAPOLIS, M…",1167,"""QUINDIO""",1,"""140410 090420 …",1,"""2563000 682250…",1,"""EVERWIN MARITI…",1,"""MMIV""",1,null,null
null,null,"""HPU""",1,"""WESTERN SAHARA…",6,"""NORTH AMERICA""",722520,"""57035""",1,"""FREDERIKSTED""",1,"""78900""",1,"""TABOGUILLA IS""",1,null,null,"""GULF""",10576612,"""DULUTH MINNESO…",1423,"""VOLGODONSK""",1,"""732399 950210 …",1,"""3956000 727500…",1,"""PALMDALE INVES…",1,"""AVAM""",1,null,null
null,null,"""PC""",1,"""MONTENEGRO""",6,null,871385,"""91145""",1,"""CRUZ BAY""",1,"""77051""",1,"""VITYAZ""",1,null,null,"""EAST""",74496467,"""NEW YORK CITY,…",2877,"""LIGUGE""",1,"""610453 620463 …",1,"""1140000 657203…",1,"""M TWO M (M2M) …",1,"""IBIS""",1,null,null
null,null,"""QBL""",1,"""NIGER""",6,"""OCEANIA""",1597398,"""57047""",1,"""SANDUSKY""",1,"""16101""",1,"""FONTENAY LE C""",1,null,null,"""WEST""",92073665,"""BUFFALO, NEW Y…",3042,"""LENDORF""",1,"""009669 330210 …",1,"""6494000 700855…",1,"""VICTORY TRDG C…",1,"""PBAY""",1,null,null
null,null,"""BUN""",1,"""ST PIERRE AND …",7,"""CARIBBEAN""",2718638,"""24741""",1,"""BOCA BEC""",1,"""76491""",1,"""CHINNAMPO""",1,null,null,null,null,"""SAN JUAN, PUE…",6825,"""SUDHEIM""",1,"""009669 940540 …",1,"""7714216 540710…",1,"""AGROEXPORT LTD…",1,"""HPCP""",1,null,null
null,null,"""LL""",2,"""CANTON AND END…",12,"""EAST COAST SOU…",3047243,"""22556""",1,"""HAINA""",1,"""76299""",1,"""SALON DE PROV""",1,null,null,null,null,"""LAREDO, TEXAS""",8234,"""COVOLO""",1,"""110620 121190 …",1,"""6602200 146400…",1,"""KASY LOGISTICS…",1,"""VTME""",1,null,null
null,null,"""HBL""",2,"""GUINEA-BISSAU""",15,"""WEST COAST SOU…",4142811,"""13829""",1,"""SANTA RITA BR""",1,"""54929""",1,"""BIEN HOA""",1,null,null,null,null,"""MILWAUKEE, WIS…",9680,"""PT DICKSON""",1,"""848190 961610""",1,"""4633560 490320…",1,"""ZEUS LINES MAN…",1,"""SOLF""",1,null,null


In [8]:
#Counts of top exports categorical columns

#get categorical cols
exports_catcols_lzdf = (
    exports_lzdf.select(pl.col(pl.Categorical))
)
#init top cats df
exports_topcats_df = pl.DataFrame()

for colname in exports_catcols_lzdf.columns:
    col_counts_df = (
        #select column
        exports_catcols_lzdf.select(pl.col(colname))
        .collect()
        .to_series()
        #get top 10 value counts
        .value_counts()
        .sort('count', descending=True)
        .limit(10)
        #rename count column
        .rename({'count': colname + '_count'})
    )
    #stack dfs horizontally 
    exports_topcats_df = pl.concat([exports_topcats_df, col_counts_df], how='horizontal')

print('Counts of top 10 categories for each categorical variable in exports database:')
exports_topcats_df

Counts of top 10 categories for each categorical variable in exports database:


weight_unit,weight_unit_count,quantity_type,quantity_type_count,carrier_name,carrier_name_count,carrier_scac,carrier_scac_count,departure_port_code,departure_port_code_count,departure_port_name,departure_port_name_count,coast_region,coast_region_count,dest_territory,dest_territory_count,dest_region,dest_region_count,dest_port_name,dest_port_name_count
cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32,cat,u32
"""KG""",68769489,"""CF""",20052121,"""MEDITERRANEAN …",5793495,"""HAPL""",4861095,"""4601""",9016653,"""NEW YORK""",9016653,"""EAST""",41638045,"""CHINA (MAINLAN…",5577757,"""NORTH EAST ASI…",17045863,"""SAN JUAN""",3911708
null,null,null,6958696,"""HAPAG LLOYD""",4861294,"""MLSL""",3539362,"""2704""",6071495,"""LOS ANGELES""",6071495,"""WEST""",18675268,"""PUERTO RICO""",3941260,"""CARIBBEAN""",13873527,"""BUSAN""",2606295
null,null,"""UNT""",6389123,"""MAERSK LINE""",4856652,"""MSCU""",3280334,"""5301""",5271211,"""HOUSTON""",5271206,"""GULF""",8002937,"""JAPAN""",3796480,"""NORTH EUROPE""",8987532,"""HONG KONG""",2430256
null,null,"""PCS""",6019382,"""CMA-CGM""",3296996,"""CMDU""",2930458,"""2709""",5249455,"""LONG BEACH""",5249455,"""CARIBBEAN""",436480,"""REPUBLIC OF KO…",3083205,"""CENTRAL AMERIC…",5586530,"""SHANGHAI""",2193297
null,null,"""PKG""",4463101,"""SEABOARD MARIN…",3223745,"""SMLU""",2575867,"""1803""",4646304,"""JACKSONVILLE""",4646304,"""GREAT LAKES""",16283,"""HONG KONG""",2430461,"""SOUTH EAST ASI…",3902742,"""ANTWERP""",2192089
null,null,"""CTN""",2784205,"""TROPICAL SHIPP…",2818095,"""MDSC""",2513240,"""5203""",4522768,"""PT EVERGLADES""",4522768,null,476,"""BELGIUM""",2306736,"""MEDITERRANEAN""",3486244,"""KAOHSIUNG""",1658597
null,null,"""PLT""",2730319,"""ORIENT OVERSEA…",2720267,"""TRSL""",2308826,"""1703""",4481005,"""SAVANNAH""",4481005,null,null,"""BAHAMAS""",2299915,"""AFRICA""",3342850,"""ROTTERDAM""",1619013
null,null,"""CS""",2713884,"""CROWLEY LINER …",2705052,"""CAMN""",2102643,"""5201""",4120195,"""MIAMI""",4120195,null,null,"""TAIWAN""",2156880,"""WEST COAST SOU…",2615873,"""BREMERHAVEN""",1433094
null,null,"""CM""",2504807,"""EVERGREEN LINE…",2316248,"""EGLV""",2093878,"""1401""",4083423,"""NORFOLK""",4083423,null,null,"""GERMANY""",2073918,"""EAST COAST SOU…",2601738,"""TOKYO""",1408627


## Exploratory Stats and Visuals

## Known Data Issues

The below cells inspect known, unresolved issues in the database such as duplicated Bills of Lading, inconsistent carrier names/codes, etc. 

### Duplicate BOLs 

In [9]:
#count unique bol_scac IDs
import_bols_unique_n = imports_lzdf.select(pl.col('bol_id')).unique().select(pl.count()).collect().item()
export_bols_unique_n = exports_lzdf.select(pl.col('bol_id')).unique().select(pl.count()).collect().item()

print('{:,} out of {:,} rows ({:.2f}%) in the imports dataset contain duplicated BoLs.'.format(imports_n-import_bols_unique_n, imports_n, (imports_n-import_bols_unique_n)/imports_n*100))
print('{:,} out of {:,} rows ({:.2f}%) in the exports dataset contain duplicated BoLs.'.format(exports_n-export_bols_unique_n, exports_n, (exports_n-export_bols_unique_n)/exports_n*100))

1,158,525 out of 178,875,896 rows (0.65%) in the imports dataset contain duplicated BoLs.
3,488,195 out of 68,769,489 rows (5.07%) in the exports dataset contain duplicated BoLs.


Possible reasons:
- data entry errors
- aggregations of some kind by S&P
- ???

### Missing weight and TEU data 

- weight data is inconsisent prior to 2017
    - inexplicably spiking 2015-2017 
    - inexplicably low 2013-2014
    - entirely missing prior to 2013

- TEU data is similarly strange
    - large jump 2021-2022
    - inexplicably low 2013-2014
    - entirely missing prior to 2013

We are pursuing this with S&P

### Carrier Names 

In [10]:
carriers_df = (imports_lzdf.select([pl.col('carrier_scac'), 
                                    pl.col('carrier_name')])
               .unique() 
               .sort('carrier_scac', descending=True)
               .collect()
               )

scac_unique = (
    imports_lzdf.select(pl.col('carrier_scac'))
    .unique()
    .cast(pl.Utf8)
    .sort('carrier_scac')
    .collect()
)
#scac_unique.write_csv('scac_unique.csv')

In [11]:
carriers_df.describe()

describe,carrier_scac,carrier_name
str,str,str
"""count""","""4383""","""3993"""
"""null_count""","""1""","""391"""
"""mean""",null,null
"""std""",null,null
"""min""",null,null
"""25%""",null,null
"""50%""",null,null
"""75%""",null,null
"""max""",null,null


In [12]:
dupscacs_df = (
    carriers_df.drop_nulls(subset='carrier_name')
    .filter(pl.col('carrier_scac').is_duplicated())
)

dupnames_df = (
    carriers_df.drop_nulls(subset='carrier_scac')
    .filter(pl.col('carrier_name').is_duplicated())
    .sort('carrier_name')
)

Discussion:
- There appear to be very few (~200 out of 178M) duplications of SCAC codes based on different spelling or naming of carriers.
- SCAC codes that need to be addressed:
    - 'BULK' (~50 rows)
    - '-1' (1 row)
    - 'ZZZZ' (a carrier scac corresponding to entirely missing carrier name data in this database)
    - these could be coded as null values in the ETL step

# Older code - likely to be dropped

## Value and Volumes by Year:

In [ ]:
#get year col
pldf['year'] = pldf.date_arrival.dt.to_period('Y')
#group value and volume by year
activityacrosstime_df = pldf[['year', 'teus', 'value_est']].groupby('year').sum()
#plot
sns.barplot(data=activityacrosstime_df, x='year', y='value_est');
plt.title('Total Value of Imports Over Time')
plt.xticks(rotation=45);

In [ ]:
#plot
sns.barplot(data=activityacrosstime_df, x='year', y='teus');
plt.title('Total Volume (TEUs) of Imports Over Time')
plt.xticks(rotation=45);

I guess value and volume records weren't kept before ~2015 ?!?

In [ ]:
#group value and volume by year
until2012_df = pldf[pldf['year']< pd.Period(2013)]
until2012_df = until2012_df[['year', 'teus', 'value_est']].groupby('year').sum()
#plot
sns.barplot(data=until2012_df, x='year', y='teus');
plt.title('Total Volume (TEUs) of Imports Over Time')
plt.xticks(rotation=45);

Must be missing data here?

In [ ]:
#get year col
pldf['year'] = pldf.date_arrival.dt.to_period('Y')
#group value and volume by year
activityacrosstime_df = pldf[['year', 'container_piece_count']].groupby('year').sum()
#plot
sns.barplot(data=activityacrosstime_df, x='year', y='container_piece_count');
plt.title('Total Volume (Container Piece Count) of Imports Over Time')
plt.xticks(rotation=45);

In [ ]:
pldf.head()

## Carriers Over Time

In [ ]:
carriersovertime_df = pldf[['year', 'carrier_scac']].groupby('year').nunique()

In [ ]:
sns.barplot(data=carriersovertime_df, x='year', y='carrier_scac');
plt.title('Number of Unique Carriers Over Time');
plt.xticks(rotation=45);

Was there really a spike in carriers in 2010 and 2012? Or does this indicate changes in the way SCAC codes are assigned?

### Market share of the 50 largest carriers by estimated value 

In [ ]:
#get largest carriers
carriers_df = pldf[pldf.year > pd.Period(2014)]
carriers_df = carriers_df[['year', 'carrier_scac', 'value_est']].groupby(['year', 'carrier_scac']).sum()

In [ ]:
carriers_df.columns = ['value_usd']
carriers_df.sort_values('value_usd', ascending=False, inplace=True)
carriers_df.sort_values('year', inplace=True)

In [ ]:
carriers_df.reset_index(inplace=True)
carriers_df.head()